In [89]:
import os, re, mmap, random

unigramfile="unigramfile.dat"
bigramfile="bigramfile.dat"
trigramfile="trigramfile.dat"
quadgramfile="quadgramfile.dat"

with open(unigramfile, 'rb', 0) as file, \
    mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as text:
    unigramtotal= text.read(text.find(b'[')).decode('utf-8')
    unigrams= text.read(text.find(b']')).decode('utf-8')

with open(bigramfile, 'rb', 0) as file, \
    mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as text:
    bigramtotal= text.read(text.find(b'[')).decode('utf-8')
    bigrams= text.read(text.find(b']')).decode('utf-8')

with open(trigramfile, 'rb', 0) as file, \
    mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as text:
    trigramtotal= text.read(text.find(b'[')).decode('utf-8')
    trigrams= text.read(text.find(b']')).decode('utf-8')
    
with open(quadgramfile, 'rb', 0) as file, \
    mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as text:
    quadgramtotal= text.read(text.find(b'[')).decode('utf-8')
    quadgrams= text.read(text.find(b']')).decode('utf-8')

words=unigrams.replace('[','').replace(']','').replace('(','').replace(')','').split(',')
unigramrange=0
for w in range(1,len(words)-1,2):
    unigramrange +=int(words[w])

def generate(seedtext="the",
             length=100,
             weight1=1,
             weight2=2,
             weight3=3,
             weight4=4, 
            ):

    if length==0:
        output=''
    print("   Here is what we are doing. Scroll down for the final result.\n")
    for gword in range(1,length+1):
        if gword > 3:                     # use quadgram model

            print(f"Searching quadigrams for '{currenttrigram}'.")  
            if quadgrams.find("'" + currenttrigram + ' ') > -1:
                nextword = quadgrams[quadgrams.find("'" + currenttrigram + ' ')+len(currenttrigram)+2:quadgrams.find("'",quadgrams.find(currenttrigram + ' '))]
                print("  Found the most frequently used next word.")
            else:
                print(f"    Not found. Searching trigrams for '{currentbigram}'.")  
                if trigrams.find("'" + currentbigram + ' ') > -1:
                    nextword = trigrams[trigrams.find("'" + currentbigram + ' ')+len(currentbigram)+2:trigrams.find("'",trigrams.find(currentbigram + ' '))]
                    print("      Found the most frequently used next word.")
                else:
                    print(f"        Not found. Searching bigrams for '{currentword}'.")        
                    if bigrams.find("'" + currentword + ' ') > -1:
                        nextword = bigrams[bigrams.find("'" + currentword + ' ')+len(currentword)+2:bigrams.find("'",bigrams.find(currentword + ' '))]
                        print("          Found the most frequently used next word.")
                    else:
                        rand=random.randint(0,unigramrange)
                        look = rand
                        for w in range(1,len(words)-1,2):
                            look = look - int(words[w])
                            if look < 0:
                                nextword = words[w-1][2:-1]
                                break
                        print(f"            Not found. We randomly choose '{nextword}'.")
            pastword =currentword
            currentword=nextword
            currentquadgram=currenttrigram + ' ' + currentword
            currenttrigram=currentbigram + ' ' + currentword
            currentbigram=pastword + ' ' + currentword
            output+=' ' + currentword 
            
            
        else:
            if gword == 3:                 # use trigram model
                print(f"    Searching trigrams for '{currentbigram}'.")  
                if trigrams.find("'" + currentbigram + ' ') > -1:
                    nextword = trigrams[trigrams.find("'" + currentbigram + ' ')+len(currentbigram)+2:trigrams.find("'",trigrams.find(currentbigram + ' '))]
                    print("      Found the most frequently used next word.")
                else:
                    print(f"        Not found. Searching bigrams for '{currentword}'.")        
                    if bigrams.find("'" + currentword + ' ') > -1:
                        nextword = bigrams[bigrams.find("'" + currentword + ' ')+len(currentword)+2:bigrams.find("'",bigrams.find(currentword + ' '))]
                        print("          Found the most frequently used next word.")
                    else:
                        rand=random.randint(0,unigramrange)
                        look = rand
                        for w in range(1,len(words)-1,2):
                            look = look - int(words[w])
                            if look < 0:
                                nextword = words[w-1][2:-1]
                                break
                        print(f"            Not found. We randomly choose '{nextword}'.")
                pastword =currentword
                currentword=nextword
                currenttrigram=currentbigram + ' ' + currentword
                currentbigram=pastword + ' ' + currentword
                output+=' ' + currentword 

                
            elif gword == 2:                 # use bigram model
                print(f"        Searching bigrams for '{currentword}'.")        
                if bigrams.find("'" + currentword + ' ') > -1:
                    nextword = bigrams[bigrams.find("'" + currentword + ' ')+len(currentword)+2:bigrams.find("'",bigrams.find(currentword + ' '))]
                    print("          Found the most frequently used next word.")
                else:
                    rand=random.randint(0,unigramrange)
                    look = rand
                    for w in range(1,len(words)-1,2):
                        look = look - int(words[w])
                        if look < 0:
                            nextword = words[w-1][2:-1]
                            break
                    print(f"            Not found. We randomly choose '{nextword}'.")
                pastword =currentword
                currentword=nextword
                currentbigram=pastword + ' ' + currentword
                output+=' ' + currentword

            elif gword == 1:                     # check seedtext
                for char in range(0,len(seedtext)):
                    maybe = seedtext[:len(seedtext)-char]
                    print(f"            Searching unigrams for '{maybe}'.")        
                    if unigrams.find(maybe) > -1:
                        if unigrams[unigrams.find(maybe)-1] == "'":
                            print("              Found the word (or a word that starts with it).")
                            currentword = unigrams[unigrams.find(maybe):unigrams.find("'",unigrams.find(maybe))]
                            break
                    print("              Not found. Dropping a letter")
                    currentword = ""           
                if currentword == "":
                    rand=random.randint(0,unigramrange)
                    look = rand
                    for w in range(1,len(words)-1,2):
                        look = look - int(words[w])
                        if look < 0:
                            currentword = words[w-1][2:-1]
                            break
                    print(f"                We randomly choose '{currentword}'.")
                output=currentword
    
    print(f"\n\n     Given '{seedtext}', our initial model generates the following {length} words:\n\n{output.replace(' .','.')}")
      
generate()

   Here is what we are doing. Scroll down for the final result.

            Searching unigrams for 'the'.
              Found the word (or a word that starts with it).
        Searching bigrams for 'the'.
          Found the most frequently used next word.
    Searching trigrams for 'the same'.
      Found the most frequently used next word.
Searching quadigrams for 'the same time'.
  Found the most frequently used next word.
Searching quadigrams for 'same time the'.
  Found the most frequently used next word.
Searching quadigrams for 'time the most'.
    Not found. Searching trigrams for 'the most'.
      Found the most frequently used next word.
Searching quadigrams for 'the most part'.
  Found the most frequently used next word.
Searching quadigrams for 'most part of'.
  Found the most frequently used next word.
Searching quadigrams for 'part of the'.
  Found the most frequently used next word.
Searching quadigrams for 'of the country'.
  Found the most frequently used next word.
S

In [97]:
generate("?",25)

   Here is what we are doing. Scroll down for the final result.

            Searching unigrams for '?'.
              Not found. Dropping a letter
                We randomly choose 'am'.
        Searching bigrams for 'am'.
          Found the most frequently used next word.
    Searching trigrams for 'am not'.
      Found the most frequently used next word.
Searching quadigrams for 'am not a'.
  Found the most frequently used next word.
Searching quadigrams for 'not a man'.
  Found the most frequently used next word.
Searching quadigrams for 'a man to'.
  Found the most frequently used next word.
Searching quadigrams for 'man to be'.
  Found the most frequently used next word.
Searching quadigrams for 'to be a'.
  Found the most frequently used next word.
Searching quadigrams for 'be a very'.
  Found the most frequently used next word.
Searching quadigrams for 'a very good'.
  Found the most frequently used next word.
Searching quadigrams for 'very good one'.
    Not found. Searching

In [67]:
generate("why")

   Here is what we are doing. Scroll down for the final result.

            Searching unigrams for 'why'.
              Found the word (or a word that starts with it).
        Searching bigrams for 'why'.
          Found the most frequently used next word.
    Searching trigrams for 'why I'.
      Found the most frequently used next word.
Searching quadigrams for 'why I should'.
  Found the most frequently used next word.
Searching quadigrams for 'I should not'.
  Found the most frequently used next word.
Searching quadigrams for 'should not have'.
  Found the most frequently used next word.
Searching quadigrams for 'not have been'.
  Found the most frequently used next word.
Searching quadigrams for 'have been more'.
  Found the most frequently used next word.
Searching quadigrams for 'been more than'.
  Found the most frequently used next word.
Searching quadigrams for 'more than once'.
  Found the most frequently used next word.
Searching quadigrams for 'than once .'.
    Not found